# ОБРАБОТКА ДАННЫХ:
##### Были скачаны файлы со статистикой по различным тематикам сообществ и был реализован алгоритм в Python, в котором считываются файлы из директории и обрабатываются с целью формирования матрицы объекты-признаки.


## 1. Инициализация первичных данных из файлов со статистикой

In [1]:
import pandas as pd
import glob
import numpy as np
import os.path
import random
from collections import defaultdict

tegs = ['Дни недели','Время суток',
      'Лайки','Репосты','Комментарии','Просмотры','ER',
      'Подписчики','Дни недели/объем текста',
      'Время суток/объем текста','Количество по длине текста',
      'ER по длине текста','Количество по типу контента','ER по типу контента',
      'Количество публикаций','Хештеги','Хештеги/ER',
      'Относительная активность дни недели/объем текста',
      'Относительная активность время суток/объем текста',
      'Относительная активность по типу контента',
      'Относительная активность по длине текста',
      'Относительная активность по хештегам']
tegs_pd = pd.DataFrame(tegs)
tegs_extra = ['Относительная активность дни недели/объем текста',
      'Относительная активность время суток/объем текста',]
tegs_extra_pd = pd.DataFrame(tegs_extra)
tegs_full = ['Сравнительная таблица','Дни недели','Время суток',
      'Лайки','Репосты','Комментарии','Просмотры','ER',
      'Подписчики','Дни недели/объем текста',
      'Время суток/объем текста','Количество по длине текста',
      'ER по длине текста','Количество по типу контента','ER по типу контента',
      'Количество публикаций','Хештеги','Хештеги/ER',
      'Относительная активность дни недели/объем текста',
      'Относительная активность время суток/объем текста',
      'Относительная активность по типу контента',
      'Относительная активность по длине текста',
      'Относительная активность по хештегам']
tegs_full_pd=pd.DataFrame(tegs_full)
m=0;n=0
df_1=pd.DataFrame();df_2=pd.DataFrame();df_3=pd.DataFrame();df_4=pd.DataFrame();df_5=pd.DataFrame();df_6=pd.DataFrame()
df_7=pd.DataFrame();df_8=pd.DataFrame();df_9=pd.DataFrame();df_10=pd.DataFrame();df_11=pd.DataFrame();df_12=pd.DataFrame()
df_13=pd.DataFrame();df_14=pd.DataFrame();df_15=pd.DataFrame();df_16=pd.DataFrame();df_17=pd.DataFrame();df_18=pd.DataFrame()
df_19=pd.DataFrame();df_20=pd.DataFrame();df_21=pd.DataFrame();df_22=pd.DataFrame();df_23=pd.DataFrame()

day_week=['Пн','Вт','Ср','Чт','Пт','Сб','Вс']
day_week_pd=pd.DataFrame(day_week)
day_week_pd=day_week_pd.T

def record_excel(df,str):
    writer = pd.ExcelWriter(str + '.xlsx', engine='xlsxwriter')
    df.to_excel(writer, sheet_name='Sheet2')
    writer.save()

## 2. Формирование фреймов данных по характеристикам (признакам)

In [2]:
path = 'D:\Projects_for_my_science_work\Article_1_classification\data_of_community\data_vk\Vk_data_all_type_test_3'
num_files = len([f for f in os.listdir(path)
                if os.path.isfile(os.path.join(path, f))])
print(num_files)

count_file=0
for f in glob.glob('D:\Projects_for_my_science_work\Article_1_classification\data_of_community\data_vk\Vk_data_all_type_test_3\*.xlsx'):
    print(f)
    count_file=count_file+1
    df = pd.read_excel(f, index_col=None, header=None, usecols=None)

    # 1. СЧИТЫВАНИЕ И СРАЗУ ЖЕ ОБРАБАТЫВАНИЕ ФАЙЛА БЕЗ ПРОПУСКОВ
    df.iloc[1][0]=df.iloc[0][0]
    df.iloc[0][0]=None
    print(len(df.index))
    for j in range(0,len(tegs_pd)):
         for i in range(0,len(df.index)-1):
            if df.iloc[i][0]==tegs_pd.iloc[j][0]:
                df.iloc[i+1][0]=df.iloc[i][0]
                df.iloc[i][0]=None
                break
    df.dropna(how='all', inplace=True)

    # 2. Определение количества выгруженных сообществ в каждом отдельном файле identification_community
    identification_community = pd.DataFrame(np.where(df[0]=='Дни недели'))-pd.DataFrame(np.where(df[0]=='Сравнительная таблица'))-1

    # 3. копирую dataframe, чтобы сформировать "чистый" фрейм без пустых строк и правильно заполненных фреймов по характеристикам
    df_clean = df.copy()
    df_clean = df_clean.replace({np.nan: None})

    # 4. Нормализация формата вывода данных у двух признаков
    for i in range(0,len(df_clean.index) - 1):
        k = 0
        if df_clean.iloc[i][0] == tegs_extra_pd.iloc[0][0] or df_clean.iloc[i][0] == tegs_extra_pd.iloc[1][0]:
            k = identification_community.iloc[0][0]
            df_copy = pd.DataFrame(index=pd.RangeIndex(1),columns=pd.RangeIndex(k))
            print(k)
            for j in range(0,k):
                df_copy.iloc[0][j] = df_clean.iloc[i][j+1]
                df_clean.iloc[i][1]=None
            for j in range(0,k):
                df_clean.iloc[i][j + 2] = df_copy.iloc[0][j]
            for j in range(0,k):
                df_clean.iloc[i][k + j + 3] = df_copy.iloc[0][j]
            for j in range(0,k):
                df_clean.iloc[i][2*k + j + 4] = df_copy.iloc[0][j]

    # 5. Определение кол-ва строк по каждой характеристике identification_deference
    iden_defer = pd.DataFrame(index=np.arange(len(tegs_full_pd.index)-1), columns=pd.RangeIndex(1))
    iden_defer = iden_defer.fillna(0)
    for i in range(1,len(tegs_full_pd.index)):
        iden_defer.iloc[i-1,:] = pd.DataFrame(np.where(df[0] == tegs_full_pd.iloc[i][0]))-pd.DataFrame(np.where(df[0] == tegs_full_pd.iloc[i-1][0])) - 1
    iden_defer = iden_defer.transpose()
    print('identification_deference')
    print(iden_defer)

    # 6. Алгоритм замены повторяющихся значений в названии сообществ
    def rename_duplicates ( iden_defer, df_clean):
        for i in range( 0, iden_defer.iloc[0][0] - 1) :
            for j in range( i + 1, iden_defer.iloc[0][0]) :
                if df_clean.iloc[i][0] == df_clean.iloc[j][0] :
                    df_clean.iloc[j][0] = str(df_clean.iloc[j][0]) + '_' + str(j)
        return df_clean

    def rename_duplicates_1 ( iden_defer, df_clean):
        for i in range( 0, iden_defer.iloc[0][0] - 1) :
            for j in range( i + 1, iden_defer.iloc[0][0]) :
                if df_clean.iloc[i][0] == df_clean.iloc[j][0] :
                    df_clean.iloc[j][0] = str(df_clean.iloc[j][0]) + '_' + str(j-1)
        return df_clean

    def rename_duplicates_2 ( df_clean ):
        for i in range( 0, len( df_clean.columns ) - 1) :
            for j in range( i + 1, len( df_clean.columns )) :
                if df_clean.iloc[0][i] == df_clean.iloc[0][j] :
                    df_clean.iloc[0][j] = str(df_clean.iloc[0][j]) + '_' + str(j-1)
        return df_clean

    def rename_duplicates_3 ( df_clean ):
        for i in range( 0, len( df_clean.columns ) - 1) :
            for j in range( i + 1, len( df_clean.columns ) ) :
                if df_clean.iloc[0][i] == df_clean.iloc[0][j] :
                    df_clean.iloc[0][j] = str(df_clean.iloc[0][j]) + '_' + str(j)
        return df_clean

    # 7. Функции для обработки различных признаков
    def func_1( i , j, df_clean, iden_defer, df ) :
        cut = df_clean.iloc[ i : i + iden_defer.iloc[0][j] + 1,].to_numpy()
        cut = pd.DataFrame( cut )
        cut = rename_duplicates( iden_defer, cut)
        df = df.append( cut )
        return df

    def func_1_1( i , j, df_clean, iden_defer, df, num_files, count_file ) :
        cut = df_clean.iloc[ i + 1 : i + iden_defer.iloc[0][j] + 1,].to_numpy(); cut = pd.DataFrame( cut )
        cut = rename_duplicates( iden_defer, cut )
        df = df.append( cut )
        if count_file == num_files:
            df = df.drop_duplicates(subset=[0], ignore_index=True)
            df = df.to_numpy(); df = pd.DataFrame(df)
        return df

    def func_2 (i , j, df_clean, iden_defer, df) :
        cut = df_clean.iloc[ i : i + iden_defer.iloc[0][j] + 1, ].to_numpy()
        cut = cut.T
        cut = np.delete( cut, (0), axis=0 )
        cut = pd.DataFrame( cut )
        cut.dropna( how='all', inplace=True )
        cut = rename_duplicates( iden_defer, cut)
        df = df.append(cut)
        df = df.drop_duplicates(subset=[0], ignore_index=True)
        return df

    def func_3 (i , j, df_clean, iden_defer, df, num_files, count_file) :
        cut = df_clean.iloc[ i : i + iden_defer.iloc[0][j] + 1, ].to_numpy(); cut = pd.DataFrame( cut )
        cut.dropna( how='all', axis=1, inplace=True )
        cut = rename_duplicates_2( cut )
        df = df.merge( cut, on=0, how='outer' )
        if count_file == num_files:
            df = df.T
            df = df.to_numpy(); df = pd.DataFrame(df)
            df = df.drop_duplicates(subset=[0], ignore_index=True)
            df = df.T
            df = df.to_numpy(); df = pd.DataFrame(df)
        return df

    def func_4 (i , j, df_clean, iden_defer, df, num_files, count_file) :
        cut = df_clean.iloc[ i : i + iden_defer.iloc[0][j] + 1, ].to_numpy()
        cut = cut.T
        cut = pd.DataFrame(cut)
        index_len=len(cut.columns)-1

        add_10 = pd.DataFrame(index = np.arange(iden_defer.iloc[0][0]+2), columns = pd.RangeIndex(len(cut.columns)*3-2))
        add_10_1 = pd.DataFrame(index = np.arange(iden_defer.iloc[0][0]+2), columns = pd.RangeIndex(len(cut.columns)))
        add_10_2 = pd.DataFrame(index = np.arange(iden_defer.iloc[0][0]+2), columns = pd.RangeIndex(len(cut.columns)))
        add_10_3 = pd.DataFrame(index = np.arange(iden_defer.iloc[0][0]+2), columns = pd.RangeIndex(len(cut.columns)))

        a = cut.iloc[ (2 + iden_defer.iloc[0][0]) : (3 + 2*iden_defer.iloc[0][0]),].to_numpy(); a = pd.DataFrame(a)
        b = cut.iloc[ (2 + 2*iden_defer.iloc[0][0] + 1) : (3 + 3*iden_defer.iloc[0][0] + 1),].to_numpy(); b = pd.DataFrame(b)
        c = cut.iloc[ (0) : (2 + iden_defer.iloc[0][0]),].to_numpy(); c = pd.DataFrame(c)


        add_10_1=add_10_1.append(c)
        add_10_1.dropna(how='all', inplace=True)
        for i in range(0,index_len):
            add_10_2.iloc[0][1 + i] = add_10_1.iloc[0][1 + i]
            add_10_3.iloc[0][1 + i] = add_10_1.iloc[0][1 + i]

        add_10_2=add_10_2.append(a); add_10_2.dropna(how='all', inplace=True)
        add_10_3=add_10_3.append(b); add_10_3.dropna(how='all', inplace=True)
        add_10=add_10.append(add_10_1); add_10.dropna(how='all', inplace=True)

        add_10 = add_10.to_numpy(); add_10 = pd.DataFrame(add_10)
        add_10_2 = add_10_2.to_numpy(); add_10_2 = pd.DataFrame(add_10_2)
        add_10_3 = add_10_3.to_numpy(); add_10_3 = pd.DataFrame(add_10_3)

        for i in range(0,iden_defer.iloc[0][0]+2):
            for j in range(0,index_len):
                add_10.iloc[i][j + index_len + 1] = add_10_2.iloc[i][j + 1]
                add_10.iloc[i][j + 2*index_len + 1] = add_10_3.iloc[i][j + 1]
        add_10 = add_10.drop( [0,1] )
        add_10 = rename_duplicates( iden_defer, add_10)
        df = df.append(add_10)
        if count_file == num_files:
            df = df.drop_duplicates(subset=[0], ignore_index=True)
            df = df.to_numpy(); df = pd.DataFrame(df)
        return df

    def func_4_1 (i , j, df_clean, iden_defer, df) :
        cut = df_clean.iloc[ i : i + iden_defer.iloc[0][j] + 1, ].to_numpy()
        cut = cut.T
        cut = pd.DataFrame(cut)
        index_len=len(cut.columns)-1

        add_10 = pd.DataFrame(index = np.arange(iden_defer.iloc[0][0]+2), columns = pd.RangeIndex(len(cut.columns)*3-2))
        add_10_1 = pd.DataFrame(index = np.arange(iden_defer.iloc[0][0]+2), columns = pd.RangeIndex(len(cut.columns)))
        add_10_2 = pd.DataFrame(index = np.arange(iden_defer.iloc[0][0]+2), columns = pd.RangeIndex(len(cut.columns)))
        add_10_3 = pd.DataFrame(index = np.arange(iden_defer.iloc[0][0]+2), columns = pd.RangeIndex(len(cut.columns)))

        a = cut.iloc[ (2 + iden_defer.iloc[0][0]) : (3 + 2*iden_defer.iloc[0][0]),].to_numpy(); a = pd.DataFrame(a)
        b = cut.iloc[ (2 + 2*iden_defer.iloc[0][0] + 1) : (3 + 3*iden_defer.iloc[0][0] + 1),].to_numpy(); b = pd.DataFrame(b)
        c = cut.iloc[ (0) : (2 + iden_defer.iloc[0][0]),].to_numpy(); c = pd.DataFrame(c)

        add_10_1 = add_10_1.append(c)
        add_10_1.dropna( how = 'all', inplace = True )
        for i in range(0,index_len):
            add_10_2.iloc[0][1 + i] = add_10_1.iloc[0][1 + i]
            add_10_3.iloc[0][1 + i] = add_10_1.iloc[0][1 + i]

        add_10_2=add_10_2.append(a); add_10_2.dropna(how='all', inplace=True)
        add_10_3=add_10_3.append(b); add_10_3.dropna(how='all', inplace=True)
        add_10=add_10.append(add_10_1); add_10.dropna(how='all', inplace=True)

        add_10 = add_10.to_numpy(); add_10 = pd.DataFrame(add_10)
        add_10_2 = add_10_2.to_numpy(); add_10_2 = pd.DataFrame(add_10_2)
        add_10_3 = add_10_3.to_numpy(); add_10_3 = pd.DataFrame(add_10_3)

        for i in range(0,iden_defer.iloc[0][0] + 2):
            for j in range(0,index_len):
                add_10.iloc[i][j + index_len + 1] = add_10_2.iloc[i][j + 1]
                add_10.iloc[i][j + 2*index_len + 1] = add_10_3.iloc[i][j + 1]

        add_10 = rename_duplicates_1( iden_defer, add_10)
        df = df.append(add_10).to_numpy(); df = pd.DataFrame(df)

        return df

    def func_5 (i , j, df_clean, iden_defer, df) :
        cut = df_clean.iloc[ i : i + iden_defer.iloc[0][j] + 1, ].to_numpy()
        cut = cut.T
        cut = pd.DataFrame( cut )
        cut = cut.replace({np.nan: None})
        cut.dropna( how = 'all', inplace = True)
        df.dropna( how = 'all', inplace = True)
        cut = rename_duplicates( iden_defer, cut)
        df = df.append( cut )
        return df

    def func_6 (i , j, df_clean, iden_defer, df) :
        cut = df_clean.iloc[ i : i + iden_defer.iloc[0][j] + 1, ].to_numpy()
        cut = pd.DataFrame(cut)
        cut.dropna ( how = 'all', axis = 1, inplace=True )
        cut = rename_duplicates_3 ( cut )
        df = df.append ( cut )
        return df

    def func_7 (i , j, df_clean, iden_defer, df) :
        cut = df_clean.iloc[ i : i + iden_defer.iloc[0][j] + 1, ].to_numpy()
        cut = cut.T
        cut = pd.DataFrame( cut )
        cut.dropna( how = 'all', inplace = True )
        cut = rename_duplicates( iden_defer, cut )
        df = df.append( cut )
        return df

    def func_8 (i , j, df_clean, iden_defer, df) :
        cut = df_clean.iloc[ i : i + iden_defer.iloc[0][j] + 1,].to_numpy()
        cut = pd.DataFrame( cut )
        cut.dropna( how = 'all', axis = 1, inplace = True )
        cut = rename_duplicates_3 ( cut )
        df = df.append( cut )
        return df

    def func_9 ( i, df_clean, df ) :
        cut = df_clean.iloc[i:len(df_clean.index)+1,].to_numpy(); cut = pd.DataFrame(cut)
        cut.dropna( how = 'all', axis = 1, inplace = True)
        cut = rename_duplicates_3 ( cut )
        df = df.merge( cut, on = 0, how = 'outer' )
        if count_file == num_files:
            df = df.T
            df = df.to_numpy(); df = pd.DataFrame(df)
            df = df.drop_duplicates(subset=[0], ignore_index=True)
            df = df.T
            df = df.to_numpy(); df = pd.DataFrame(df)
        return df

    def func_10 ( i , df_clean, df ) :
        cut = df_clean.iloc[i:len(df_clean.index)+1,].to_numpy(); cut = pd.DataFrame(cut)
        cut.dropna( how = 'all', axis = 1, inplace = True)
        cut = rename_duplicates_2( cut )
        df = df.append( cut )
        return df

   # 8. Цикл обработки файла, где происходит разбиение его на 23 отдельных фрейма
    # 8.1. Таким образом, что каждый фрейм будет содержать информацию по каждой характеристики, НО при этом по всем сообществам из дериктории
    for i in range( 0, len( df_clean.index ) ) :
        for j in range( 0, len( tegs_full_pd.index ) ) :
            if df_clean.iloc[i][0] == tegs_full_pd.iloc[j][0] :
                if count_file > 1 :
                    if j == 0:
                        df_1 = func_1_1 ( i, j, df_clean, iden_defer, df_1, num_files, count_file )
                    if j == 1:
                        df_2 = func_2 ( i, j, df_clean, iden_defer, df_2 )
                    if j == 2:
                        df_3 = func_2 ( i, j, df_clean, iden_defer, df_3 )
                    if j == 3:
                        df_4 = func_3 ( i, j, df_clean, iden_defer, df_4, num_files, count_file )
                    if j == 4:
                        df_5 = func_3 ( i, j, df_clean, iden_defer, df_5, num_files, count_file )
                    if j == 5:
                        df_6 = func_3 ( i, j, df_clean, iden_defer, df_6, num_files, count_file )
                    if j == 6:
                        df_7 = func_3 ( i, j, df_clean, iden_defer, df_7, num_files, count_file )
                    if j == 7:
                        df_8 = func_3 ( i, j, df_clean, iden_defer, df_8, num_files, count_file )
                    if j == 8:
                        df_9 = func_3 ( i, j, df_clean, iden_defer, df_9, num_files, count_file )
                    if j == 9:
                        df_10 = func_4 ( i, j, df_clean, iden_defer, df_10, num_files, count_file )
                    if j == 10:
                        df_11 = func_4 ( i, j, df_clean, iden_defer, df_11, num_files, count_file )
                    if j == 11:
                        df_12 = func_2 ( i, j, df_clean, iden_defer, df_12 )
                    if j == 12:
                        df_13 = func_2 ( i, j, df_clean, iden_defer, df_13 )
                    if j == 13:
                        df_14 = func_2 ( i, j, df_clean, iden_defer, df_14 )
                    if j == 14:
                        df_15 = func_2 ( i, j, df_clean, iden_defer, df_15 )
                    if j == 15:
                        df_16 = func_3 ( i, j, df_clean, iden_defer, df_16, num_files, count_file )
                    if j == 16:
                        df_17 = func_3 ( i, j, df_clean, iden_defer, df_17, num_files, count_file )
                    if j == 17:
                        df_18 = func_3 ( i, j, df_clean, iden_defer, df_18, num_files, count_file )
                    if j == 18:
                        df_19 = func_4 ( i, j, df_clean, iden_defer, df_19, num_files, count_file )
                    if j == 19:
                        df_20 = func_4 ( i, j, df_clean, iden_defer, df_20, num_files, count_file )
                    if j == 20:
                        df_21 = func_2 ( i, j, df_clean, iden_defer, df_21 )
                    if j == 21:
                        df_22 = func_2 ( i, j, df_clean, iden_defer, df_22 )
                    if j == 22:
                        df_23 = func_9 ( i, df_clean, df_23 )
                else:
                    if j == 0:
                        df_1 = func_1 ( i, j, df_clean, iden_defer, df_1)
                    if j == 1:
                        df_2 = func_5 ( i, j, df_clean, iden_defer, df_2)
                    if j == 2:
                        df_3 = func_5 ( i, j, df_clean, iden_defer, df_3)
                    if j == 3:
                        df_4 = func_6 ( i, j, df_clean, iden_defer, df_4)
                    if j == 4:
                        df_5 = func_6 ( i, j, df_clean, iden_defer, df_5)
                    if j == 5:
                        df_6 = func_6 ( i, j, df_clean, iden_defer, df_6)
                    if j == 6:
                        df_7 = func_6 ( i, j, df_clean, iden_defer, df_7)
                    if j == 7:
                        df_8 = func_6 ( i, j, df_clean, iden_defer, df_8)
                    if j == 8:
                        df_9 = func_6 ( i, j, df_clean, iden_defer, df_9)
                    if j == 9:
                        df_10 = func_4_1 ( i, j, df_clean, iden_defer, df_10 )
                    if j == 10:
                        df_11 = func_4_1 ( i, j, df_clean, iden_defer, df_11 )
                    if j == 11:
                        df_12 = func_7 ( i, j, df_clean, iden_defer, df_12 )
                    if j == 12:
                        df_13 = func_7 ( i, j, df_clean, iden_defer, df_13 )
                    if j == 13:
                        df_14 = func_7 ( i, j, df_clean, iden_defer, df_14 )
                    if j == 14:
                        df_15 = func_7 ( i, j, df_clean, iden_defer, df_15 )
                    if j == 15:
                        df_16 = func_8 ( i, j, df_clean, iden_defer, df_16 )
                    if j == 16:
                        df_17 = func_8 ( i, j, df_clean, iden_defer, df_17 )
                    if j == 17:
                        df_18 = func_8 ( i, j, df_clean, iden_defer, df_18 )
                    if j == 18:
                        df_19 = func_4_1 ( i, j, df_clean, iden_defer, df_19 )
                    if j == 19:
                        df_20 = func_4_1 ( i, j, df_clean, iden_defer, df_20 )
                    if j == 20:
                        df_21 = func_7 ( i, j, df_clean, iden_defer, df_21 )
                    if j == 21:
                        df_22 = func_7 ( i, j, df_clean, iden_defer, df_22 )
                    if j == 22:
                        df_23 = func_10 ( i, df_clean, df_23 )

3
D:\Projects_for_my_science_work\Article_1_classification\data_of_community\data_vk\Vk_data_all_type_test_3\charts_brends.xlsx
3792
10
10
identification_deference
   0   1   2    3    4    5    6    7    8   9   ...  12  13  14   15   16  \
0  10   7  24  499  496  497  499  499  286   7  ...   3   4   4  499  100   

    17  18  19  20  21  
0  100   7  19   4   3  

[1 rows x 22 columns]
D:\Projects_for_my_science_work\Article_1_classification\data_of_community\data_vk\Vk_data_all_type_test_3\charts_it.xlsx
3887
10
10
identification_deference
   0   1   2    3    4    5    6    7   8   9   ...  12  13  14   15   16  \
0  10   7  24  510  510  510  510  510  49   7  ...   3   4   4  510  100   

    17  18  19  20  21  
0  100   7  24   4   3  

[1 rows x 22 columns]
D:\Projects_for_my_science_work\Article_1_classification\data_of_community\data_vk\Vk_data_all_type_test_3\charts_science_technology.xlsx
3867
10
10
identification_deference
   0   1   2    3    4    5    6    7   8   9 

## 3.Формирование общего файла с характеристиками
### Фреймы без обработки или с "лёгкой" обработки, присоединяются полностью (для части характеристик)

In [3]:
df_1.dropna( how = 'all', axis = 1, inplace = True )
df_1.iloc[0][0] = 'Cообщества'

def func_plus_type1( df_main, df_attached ):

    df_copy = df_attached.copy(); df_main_copy = df_main.copy()
    for i in range( 1, len( df_copy.columns ) ):
        df_copy.iloc[0][i] = str( df_copy.iloc[0][0] ) + ' ' + str( df_copy.iloc[0][i] )
    df_copy.iloc[0][0] = 'Cообщества'
    df_main_copy = df_main_copy.merge( df_copy, on = 0, how = 'outer')
    df_main_copy = df_main_copy.to_numpy(); df_main_copy = pd.DataFrame( df_main_copy )

    return df_main_copy

def func_plus_type2( df_main, df_attached ):

    df_copy = df_attached.copy(); df_main_copy = df_main.copy()
    for i in range(1,len(df_copy.columns)):
        df_copy.iloc[0][i] = str( df_copy.loc[0][0] ) + ' ' + str( df_copy.loc[0][i] ) + ' ' + str( df_copy.loc[1][i] )
    df_copy = df_copy.drop( df_copy.index[[1]] )
    df_copy.iloc[0][0] = 'Cообщества'
    df_main_copy = df_main_copy.to_numpy(); df_main_copy = pd.DataFrame( df_main_copy )
    df_main_copy = df_main_copy.merge( df_copy, on = 0, how = 'outer' )
    df_main_copy = df_main_copy.to_numpy(); df_main_copy = pd.DataFrame( df_main_copy )

    return df_main_copy

In [4]:
char_2 = func_plus_type1( df_1, df_2 )
char_3 = func_plus_type1( char_2, df_3 )
char_10 = func_plus_type2( char_3, df_10 )
char_11 = func_plus_type2( char_10, df_11 )
char_12 = func_plus_type1( char_11, df_12 )
char_13 = func_plus_type1( char_12, df_13 )
char_14 = func_plus_type1( char_13, df_14 )
char_15 = func_plus_type1( char_14, df_15 )
char_19 = func_plus_type2( char_15, df_19 )
char_20 = func_plus_type2( char_19, df_20 )
char_21 = func_plus_type1( char_20, df_21 )
first_part_features = func_plus_type1( char_21, df_22 )

In [5]:
record_excel( first_part_features, 'first_part_features_article' )
print( first_part_features.iloc[:15, :15] )

                                0            1                 2   \
0                       Cообщества  Подписчиков  Всего публикаций   
1                 SNICKERS® Russia       355793                 3   
2                           Дом.ру       299401               313   
3                            Durex      3049818               364   
4                  adidas Football       354015               197   
5                     Venus Russia       604804                 0   
6                     Tele2 Россия       705121               292   
7                             Dove       128677                18   
8                              HTC       152644                 0   
9                            Intel       424152                 0   
10               Макдоналдс Россия      1505791               517   
11            Типичный программист       529819              2278   
12                    Журнал «Код»       119751              2527   
13  ITc | сообщество программистов

# Фреймы c заполнением пропусков, присоединяются только обобщённые значения для каждого сообщества:

 ### 1. Средним значением стандартный метод
 ### 2. Медианой

## Методология:
#### Пропуски заполняются по правилу:
#### Для второй группы требуются преобразования. Были реализованы 4 функции, где заполняются пропуски значений по формуле:
#### x(i,j) = E_j + rand(-2,2) × σ_j,
#### x(i,j) = Me_j + rand(-2,2) × σ_j
#### где x(i,j) – пустое значение случайной величины c номером i ̇, E_j, Me_j – математическое ожидание и медиана для j сообщества, σ_j – среднеквадратическое отклонение для j сообщества.

In [6]:
pd.options.mode.chained_assignment = None # default='warn'
def row_1_func ( df_attached ):
        df_new = df_attached.iloc[ 0,: ]
        df_new = df_new.to_numpy()
        df_new = pd.DataFrame( df_new )
        return df_new
def normalize_func ( df_attached ):
    df_attached.dropna(how = 'all', axis = 1, inplace=True)
    df_attached = df_attached.replace( {np.nan: None} )
    df_attached = df_attached.iloc[ 1 : len( df_attached.index ) , 1 : len( df_attached.columns ) ]
    df_attached = df_attached.to_numpy()
    df_attached = pd.DataFrame( df_attached )
    return df_attached

def count_zeros ( df_attached ):
    c_zeros = pd.DataFrame()
    for j in range( 0, len( df_attached.columns ) ):
        c_zeros = len(df_attached) - df_attached.count(0)
    return c_zeros

def count_nan ( df_attached ):
    c_nan = pd.DataFrame()
    for j in range( 0, len( df_attached.columns ) ):
        c_nan = len(df_attached) - df_attached.count()
    return c_nan

def insert_value ( df_attached, df_col ):
    df_attached_copy = df_attached.copy(); df_col_copy = df_col.copy()
    df_std = df_attached_copy.std(axis = 0)
    z = count_zeros ( df_attached_copy ); n = count_nan ( df_attached_copy )

    for j in range( 0, len( df_attached_copy.columns ) ):
        for i in range( 0,len( df_attached_copy.index ) ):
            if z.iloc[j] >= len(df_attached_copy.index)/2 or n.iloc[j] >= len(df_attached_copy.index)/2:
                df_attached_copy = df_attached_copy.fillna( 0 )
            else:
                if df_attached_copy.iloc[i][j] is None or df_attached_copy.iloc[i][j] == 0:
                    df_attached_copy.iloc[i][j] = df_col_copy.iloc[j][0] + random.uniform( -2, 2 ) * df_std.iloc[j]

    return df_attached_copy

# 1. Mean_mean
def func_mean_mean( df_main, df_attached ):
    df = normalize_func( df_attached )
    df_1_row = row_1_func( df_attached ).T

    df_copy_mean = df.copy()
    df4_mean = round( df.mean() )
    df4_mean = df4_mean.fillna( 0 )
    df4_mean = pd.DataFrame( df4_mean )
    y_full_mean = insert_value ( df_copy_mean, df4_mean )
    mean_mean = y_full_mean.mean()
    mean_mean = pd.DataFrame( mean_mean ).T
    char_mean_mean  = df_1_row.append( mean_mean )

    char_mean_mean = pd.concat( [char_mean_mean], ignore_index = True)
    m = char_mean_mean.iloc[ 1 , 0 : ( len(char_mean_mean.columns)-1 )].copy()
    m = m.to_numpy(); m = pd.DataFrame( m ); m = m.fillna( 0 )
    char_mean_mean.iloc[1][0] = str( char_mean_mean.iloc[0][0] ) + ' ' + 'mean mean'
    char_mean_mean.iloc[0][0] = 'Cообщества'

    for i in range( 1 , len( char_mean_mean.columns ) ):
        if char_mean_mean.iloc[1][0] == 'ER mean mean':
            char_mean_mean.iloc[1][i] = m.iloc[i-1][0]
        else:
            char_mean_mean.iloc[1][i] = round( m.iloc[i-1][0] )
    char_mean_mean = char_mean_mean.T

    df_main = df_main.merge( char_mean_mean, on = 0, how = 'outer')
    return df_main

# 2. Mean_median
def func_mean_median( df_main, df_attached ):
    df = normalize_func( df_attached )
    df_1_row = row_1_func( df_attached ).T

    df_copy_mean = df.copy()
    df4_mean = round( df.mean() )
    df4_mean = df4_mean.fillna( 0 )
    df4_mean = pd.DataFrame( df4_mean )
    y_full_mean = insert_value ( df_copy_mean, df4_mean )
    mean_median = y_full_mean.median()
    mean_median = pd.DataFrame( mean_median ).T
    char_mean_median = df_1_row.append( mean_median )

    char_mean_median = pd.concat( [ char_mean_median ], ignore_index = True)
    m = char_mean_median.iloc[ 1 , 0 : ( len(char_mean_median.columns) - 1 ) ].copy()
    m = m.to_numpy()
    m = pd.DataFrame( m )
    m = m.fillna( 0 )
    char_mean_median.iloc[1][0] = str( char_mean_median.iloc[0][0] ) + ' ' + 'mean median'
    char_mean_median.iloc[0][0] = 'Cообщества'
    for i in range( 1, len( char_mean_median.columns ) ):
        if char_mean_median.iloc[1][0] == 'ER mean median':
            char_mean_median.iloc[1][i] = m.iloc[i-1][0]
        else:
            char_mean_median.iloc[1][i] = round( m.iloc[i-1][0] )
    char_mean_median = char_mean_median.T

    df_main = df_main.merge( char_mean_median, on = 0, how = 'outer')
    return df_main

# 3. Median_Median
def func_median_median( df_main, df_attached ):
    df = normalize_func( df_attached )
    df_1_row = row_1_func( df_attached ).T

    df_copy_mean = df.copy()
    df4_median = round( df.median() )
    df4_median = df4_median.fillna( 0 )
    df4_median = pd.DataFrame( df4_median )
    y_full_mean = insert_value ( df_copy_mean, df4_median )
    mean_median = y_full_mean.median()
    mean_median = pd.DataFrame( mean_median ).T
    char_mean_median = df_1_row.append( mean_median )

    char_mean_median = pd.concat( [ char_mean_median ], ignore_index = True)
    m = char_mean_median.iloc[ 1 , 0 : ( len( char_mean_median.columns ) - 1 ) ].copy()
    m = m.to_numpy()
    m = pd.DataFrame( m )
    m = m.fillna( 0 )
    char_mean_median.iloc[1][0] = str( char_mean_median.iloc[0][0] ) + ' ' + 'median median'
    char_mean_median.iloc[0][0] = 'Cообщества'
    for i in range( 1, len( char_mean_median.columns ) ):
        if char_mean_median.iloc[1][0] == 'ER median median':
            char_mean_median.iloc[1][i] = m.iloc[i-1][0]
        else:
            char_mean_median.iloc[1][i] = round( m.iloc[i-1][0] )
    char_mean_median = char_mean_median.T

    df_main = df_main.merge( char_mean_median, on = 0, how = 'outer' )
    return df_main

# 4. Median_Mean
def func_median_mean( df_main, df_attached ):
    df = normalize_func( df_attached )
    df_1_row = row_1_func( df_attached ).T

    df_copy_mean = df.copy()
    df4_median = round( df.median() )
    df4_median = df4_median.fillna( 0 )
    df4_median = pd.DataFrame( df4_median )

    y_full_mean = insert_value ( df_copy_mean,df4_median )
    mean_median=y_full_mean.mean()
    mean_median=pd.DataFrame( mean_median ).T
    char_mean_median=df_1_row.append( mean_median )

    char_mean_median = pd.concat( [ char_mean_median ], ignore_index = True)

    m = char_mean_median.iloc[ 1 , 0 : ( len( char_mean_median.columns ) - 1) ].copy()
    m = m.to_numpy()
    m = pd.DataFrame( m )
    m = m.fillna( 0 )
    char_mean_median.iloc[1][0] = str( char_mean_median.iloc[0][0] ) + ' ' + 'median mean'
    char_mean_median.iloc[0][0] = 'Cообщества'

    for i in range( 1, len( char_mean_median.columns ) ):
        if char_mean_median.iloc[1][0] == 'ER median mean':
            char_mean_median.iloc[1][i] = m.iloc[i-1][0]
        else:
            char_mean_median.iloc[1][i] = round( m.iloc[i-1][0] )
    char_mean_median = char_mean_median.T

    df_main = df_main.merge( char_mean_median , on = 0, how = 'outer')
    return df_main

def connect_function( df_main, df_attached ):
    df_main = func_mean_mean( df_main,df_attached )
    df_main = func_mean_median( df_main, df_attached )
    df_main = func_median_median( df_main, df_attached )
    df_main = func_median_mean( df_main,df_attached )
    df_main = df_main.to_numpy()
    df_main = pd.DataFrame( df_main )
    return df_main

In [7]:
from datetime import datetime
import time
start_time = datetime.now()

char_4 = connect_function( first_part_features, df_4 )
char_5 = connect_function( char_4, df_5 )
char_6 = connect_function( char_5, df_6 )
char_7 = connect_function( char_6, df_7 )
char_8 = connect_function( char_7, df_8 )
char_9 = connect_function( char_8, df_9 )
all_features = connect_function( char_9, df_16 )

print(datetime.now() - start_time)

0:00:53.667668


In [8]:
record_excel( all_features, 'all_features_article' )
print( all_features.iloc[:15, 260:275] )

                        260                        261  \
0   Комментарии mean median  Комментарии median median   
1                         0                          0   
2                        68                         58   
3                        83                         80   
4                         0                          0   
5                         0                          0   
6                         0                          0   
7                         0                          0   
8                         0                          0   
9                         0                          0   
10                      219                        218   
11                      136                        136   
12                       70                         70   
13                      125                        125   
14                      170                        170   

                        262                  263                    264